In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [10]:
import cv2
import numpy as np
cap = cv2.VideoCapture("./running/person01_running_d1_uncomp.avi")
ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
delta=4
omega=8
hsv[...,1] = 255
while(1):
    ret, frame2 = cap.read()
    if ret==False: break
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    mag=np.floor(np.log2(np.minimum(np.maximum(mag,1.4*np.ones_like(mag)),(2**delta)*np.ones_like(mag))))*(np.uint8(np.abs(np.int16(next)-prvs))>30)
    
    ang/=np.max(ang)/omega
    ang=np.floor(ang)
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    k = cv2.waitKey(20) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('opticalfb.png',frame2)
        cv2.imwrite('opticalhsv.png',bgr)
    prvs = next
cap.release()
cv2.destroyAllWindows()

In [188]:
import cv2
import numpy as np
cap = cv2.VideoCapture("./running/person01_running_d1_uncomp.avi")

delta=4
omega=8
h=30
nt=30
ni=36 # cuboid size in x
nj=36 # cuboid size in y
hsv = np.zeros((nj,ni,3))
hsv[...,1] = 255
d=1
alpha=[0, np.pi/4, np.pi/2, 3*np.pi/4]
h_ang =[]
h_mag =[]

#for each nt block(i*nt:(i+1)*nt)
ret, frame1 = cap.read()
#for each cuboid
inx=0 #begin o the cuboid in x
iny=0 #begin o the cuboid in y
prvs = cv2.cvtColor(frame1[iny:iny+nj,inx:inx+ni],cv2.COLOR_BGR2GRAY)

for i in range(nt):
#while(1):
    ret, frame2 = cap.read()
    if ret==False: break
    next = cv2.cvtColor(frame2[iny:iny+nj,inx:inx+ni],cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    mag=np.floor(np.log2(np.minimum(np.maximum(mag,1.4*np.ones_like(mag)),(2**delta)*np.ones_like(mag))))*(np.uint8(np.abs(np.int16(next)-prvs))>h)
    ang*=(np.uint8(np.abs(np.int16(next)-prvs))>h)
    if np.max(ang)!=0.0:
        ang/=np.max(ang)/omega
        
    ang=np.floor(ang)
    
    corr_ang = greycomatrix(ang.astype(int), [d], alpha,levels=9)
    corr_mag = greycomatrix(mag.astype(int), [d], alpha,levels=4)
    print(i)
    
    haralick_ang = []
    haralick_mag = []
    for j in range(len(alpha)):
        haralick_ang.append(mahotas.features.haralick(corr_ang[:,:,0,j]))
        haralick_mag.append(mahotas.features.haralick(corr_mag[:,:,0,j]))
        #print(haralick)
        
    
    h_ang.append(haralick_ang)
    h_mag.append(haralick_mag)
      
    mag2=cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    ang2=cv2.normalize(ang,None,0,255,cv2.NORM_MINMAX)
    cv2.imshow('mag',mag2)
    cv2.imshow('ang',mag)
    k = cv2.waitKey(20) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('mag.png',mag2)
        cv2.imwrite('ang.png',mag)
    prvs = next
    

    
    
cap.release()
cv2.destroyAllWindows()

/home/alvaro/anaconda3/lib/python3.6/site-packages/skimage/feature/texture.py:109: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(image.dtype, np.float):


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [186]:
import cv2
import numpy as np
cap = cv2.VideoCapture("./running/person01_running_d1_uncomp.avi")
ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
delta=4
omega=8
hsv[...,1] = 255
#while(1):
for i in range(30):
    ret, frame2 = cap.read()
    if ret==False: break
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    mag=np.floor(np.log2(np.minimum(np.maximum(mag,1.4*np.ones_like(mag)),(2**delta)*np.ones_like(mag))))*(np.uint8(np.abs(np.int16(next)-prvs))>30)
    ang*=(np.uint8(np.abs(np.int16(next)-prvs))>30)
    if np.max(ang)!=0.0:
        ang/=np.max(ang)/omega
    ang=np.floor(ang)
    mag=cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    ang=cv2.normalize(ang,None,0,255,cv2.NORM_MINMAX)
    
    cv2.imshow('mag',mag)
    #cv2.imshow('ang',ang)
    k = cv2.waitKey(100) & 0xff
    if k == 27:
        break
    prvs = next
cap.release()
cv2.destroyAllWindows()